## imports

In [14]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

## connect to openai

In [15]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("Something is wrong. check and fix")
else:
    print("API Key found!")

API Key found!


In [16]:
openai = OpenAI()

## call to the model and check the preview

In [17]:
message = "hello!! this is my first ever message to you. How is your day?"

response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content": message}])

print(response.choices[0].message.content)

Hello! Welcome! I'm here and ready to help you. My day is going well, thank you for asking. How can I assist you today?


In [18]:
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url=url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)

In [19]:
site = Website("https://edwarddonner.com")
print(site.title)
print(site.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

### prompts

In [20]:
system_prompt = "you are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
respond in markdown"

In [21]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website called {website.title}"
    user_prompt+= "\n The content of the website is as follows; \
    please provide a concise digestable summary to the this in markdown. \
    if it includes any news or announcements, then summarize that too.\n\n"
    user_prompt+=website.text
    return user_prompt

In [22]:
system_prompt

'you are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. respond in markdown'

In [23]:
print(user_prompt_for(site))

You are looking at a website called Home - Edward Donner
 The content of the website is as follows;     please provide a concise digestable summary to the this in markdown.     if it includes any news or announcements, then summarize that too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI st

## messages

In [24]:
def messages_for(website):
    # openai's system is a list of dictionaries
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [25]:
# preview
print(messages_for(site))

[{'role': 'system', 'content': 'you are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. respond in markdown'}, {'role': 'user', 'content': 'You are looking at a website called Home - Edward Donner\n The content of the website is as follows;     please provide a concise digestable summary to the this in markdown.     if it includes any news or announcements, then summarize that too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where 

In [26]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [27]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\n**About Edward Donner**\n- Ed is a programmer and LLM (Large Language Model) enthusiast, also interested in DJing and amateur music production. \n- He is the co-founder and CTO of Nebula.io, where they apply AI to help individuals discover their potential in talent management. \n- Previously, Ed founded the AI startup untapt, which was acquired in 2021. \n- Nebula.io utilizes proprietary LLMs for talent matching and has developed a patented model, garnering significant media attention.\n\n**Courses and Events**\n- **May 28, 2025**: Launch of courses aimed at developing expertise in LLMs.\n- **May 18, 2025**: Hosting the 2025 AI Executive Briefing.\n- **April 21, 2025**: Introduction of the Complete Agentic AI Engineering Course.\n- **January 23, 2025**: Offering resources for a hands-on LLM workshop focused on agents. \n\n**Additional Information**\n- The website promotes connection opportunities via social media and email newsletters. \n- Ther

In [28]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [29]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

**About Ed:**
Edward Donner is a programmer and an enthusiast of large language models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on utilizing AI to enhance talent discovery and engagement. He has a background in starting AI companies, including untapt, which was acquired in 2021.

**Professional Interests:**
- Experimenting with LLMs
- DJing and amateur electronic music production
- Engaging with Hacker News

**Nebula.io:**
Nebula.io specializes in applying AI to help people realize their potential, primarily assisting recruiters in managing talent with innovative LLM solutions and a patented matching model.

**Recent Announcements:**
- **May 28, 2025:** Launch of a course focused on becoming an LLM expert and leader.
- **May 18, 2025:** Announcement of the 2025 AI Executive Briefing.
- **April 21, 2025:** Release of "The Complete Agentic AI Engineering Course."
- **January 23, 2025:** Workshop titled "LLM Workshop – Hands-on with Agents," which includes resources for participants.

In [32]:
# some more
display_summary("https://www.cnn.com")

# CNN Summary

### Key News Highlights

- **Trump's Administration Actions**: The White House is reviewing Smithsonian exhibits to align with President Trump's directives. Additionally, Trump's nominee for Bureau of Labor Statistics commissioner suggested suspending monthly job reports but later retracted the statement.
  
- **National Guard Deployment**: Trump has announced the deployment of National Guard soldiers in Washington, D.C., increasing federal control over local police as part of his administration's efforts in the capital.

- **Ukraine-Russia Conflict**: Reports indicate that small groups of Russian troops have broken through Ukraine's defenses in the Donetsk region as tensions continue to rise in the ongoing conflict.

- **Recent Events**:
  - The body of a missing American hiker was found in Spain's Pyrenees.
  - Taylor Swift's romantic developments with NFL star Travis Kelce are gaining media attention.
  - Kodak has warned that it may cease operations as it faces financial difficulties.

- **Climate Crisis**: Alaska is facing a glacier outburst that threatens to send a significant amount of water downstream towards Juneau, while extreme heat continues to impact Europe.

### Additional Notes
- The site encourages user feedback regarding advertising relevance and technical performance.
- Various features include links to news on health, entertainment, technology, business, sports, and global affairs with ongoing updates on various topics like the Israel-Hamas conflict and climate-related developments.

In [33]:
display_summary("https://www.anthropic.com/")

# Anthropic Overview

Anthropic focuses on developing AI technologies with an emphasis on safety and responsibility. Their main product, **Claude**, represents a series of AI models, with the latest being **Claude Opus 4.1** and **Claude Sonnet 4**. These models are designed to assist in various applications, including coding, customer support, and education, showcasing significant advancements in processing capabilities and functionalities.

## Key Announcements:
- **ISO 42001 Certification**: Anthropic has announced their certification to ISO 42001, which underscores their commitment to high standards in AI development.
- **New AI Models**: Updates include Claude Opus 4.1, aimed at handling complex tasks in coding and AI-assisted workflows.

## Core Philosophy:
Anthropic prioritizes the long-term well-being of humanity in their AI endeavors, advocating for responsible AI development and transparency in their operations. Their work includes ongoing research and policy initiatives to address societal impacts of AI technologies.